In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("2017_2019.csv")
df.drop(['Unnamed: 18'],inplace=True,axis=1)

In [4]:
dates = set()
for i,data in df.iterrows():
    dates.add(pd.to_datetime( 
                             str(data['Year'])[:-2]+'-'+str(data['Month'])[:-2] + '-' + str(data['Day'])[:-2],
                             format="%Y-%m-%d"
                             ))

In [5]:
def get_day(df:pd.DataFrame,day:pd.Timestamp) -> pd.DataFrame:
    
    return df.loc[ (df ['Year'] == day.year) & 
                  (df['Day'] == day.day) &
                  (df['Month'] == day.month)]
    
def get_records_night(df:pd.DataFrame,day:pd.Timestamp):
    records_beafore_df = get_n_days_beafore(df,day,1)
    records_beafore_df = records_beafore_df.loc[(records_beafore_df.Hour > 12)  & (records_beafore_df["Clearsky DNI"] != 0)]
    
    records_df = get_day(df,day)
    records_df = records_df.loc[(records_df.Hour < 12) &( records_df.DNI == 0) &(records_df["Clearsky DNI"] != 0)]
    
    return(pd.concat([records_beafore_df,records_df]))
    
    
  
def get_n_days_beafore(df:pd.DataFrame,day:pd.Timestamp,n:int):
    day_beafore = day - pd.offsets.Day(n)
    return get_day(df,day_beafore)

def get_working_day(df:pd.DataFrame,day:pd.Timestamp):
    records = get_day(df,day)
    return records.loc[(records.DNI != 0) | (records["Clearsky DNI"] != 0)]

    
    

In [26]:
_ = pd.Series( list(dates)).sort_values()
get_records_night(df,_.iloc[2])

,Year,Month,Day,Hour,Minute,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,DHI,DNI,GHI,Relative Humidity,Solar Zenith Angle,Surface Albedo,Pressure,Wind Speed
156,2017,1,2,15,0,5.6,0,0,0,-3.1,0,0,0,53.36,89.77,0.13,1000,0.5
157,2017,1,2,15,15,5.5,0,0,0,-3.1,0,0,0,53.73,92.90,0.13,1000,0.5
158,2017,1,2,15,30,5.4,0,0,0,-3.1,0,0,0,54.10,95.56,0.13,1000,0.5
159,2017,1,2,15,45,5.3,0,0,0,-3.1,0,0,0,54.81,98.26,0.13,1000,0.5
160,2017,1,2,16,0,5.2,0,0,0,-3.1,0,0,0,55.20,101.00,0.13,1000,0.4
161,2017,1,2,16,15,5.0,0,0,0,-3.1,0,0,0,55.97,103.79,0.13,1000,0.5
162,2017,1,2,16,30,4.9,0,0,0,-3.1,0,0,0,56.36,106.60,0.13,1000,0.6
163,2017,1,2,16,45,4.7,0,0,0,-2.8,0,0,0,58.43,109.45,0.13,1001,0.7
164,2017,1,2,17,0,4.5,0,0,0,-2.8,0,0,0,59.25,112.32,0.13,1001,0.8
165,2017,1,2,17,15,4.4,0,0,0,-2.8,0,0,0,59.66,115.22,0.13,1001,0.9


In [6]:
df[df.isna().any(axis=1)]

,Year,Month,Day,Hour,Minute,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,DHI,DNI,GHI,Relative Humidity,Solar Zenith Angle,Surface Albedo,Pressure,Wind Speed


In [7]:
records_df =pd.DataFrame()
for idx,date in enumerate( pd.Series( list(dates)).sort_values().values):
    
    date = pd.to_datetime(date)
    day = get_working_day(df,date)
    record = pd.DataFrame([{'date':date}])
    
    record['month'] = date.month
    record['len_day'] = len(day)
    record['temp_mean'] = day.Temperature.mean()
    record['press_mean'] = day.Pressure.mean()
    record['wind_mean'] = day["Wind Speed"].mean()
    
    
    night = get_records_night(df,date)
    
    if idx == 0:
        record['len_night'] = 0
    else:
        record['len_night'] = len(night)
        record['night_temp_mean'] = night.Temperature.mean()
        record['night_press_mean'] = night.Pressure.mean()
        record['night_wind_mean'] = night["Wind Speed"].mean()
    
    record['Y'] = (day["Clearsky DNI"].sum() + day['DNI'].sum()) / record['len_day']
    records_df = pd.concat([records_df,record])
records_df

,date,month,len_day,temp_mean,press_mean,wind_mean,len_night,Y,night_temp_mean,night_press_mean,night_wind_mean
0,2017-01-01,1,37,4.910811,1001.648649,1.878378,0,1397.243243,NaN,NaN,NaN
0,2017-01-02,1,37,6.035135,1000.918919,1.335135,11,1391.621622,3.863636,1000.727273,1.709091
0,2017-01-03,1,37,8.983784,999.756757,2.970270,14,1068.054054,5.828571,1000.285714,1.835714
0,2017-01-04,1,37,10.167568,996.621622,3.432432,15,685.891892,8.826667,998.400000,2.533333
0,2017-01-05,1,37,14.021622,988.594595,6.070270,26,538.972973,12.834615,990.846154,5.188462
...,...,...,...,...,...,...,...,...,...,...,...
0,2019-12-27,12,37,8.959459,996.324324,3.143243,8,1206.540541,9.762500,989.875000,3.962500
0,2019-12-28,12,37,8.727027,991.810811,1.532432,22,595.216216,8.468182,993.545455,1.740909
0,2019-12-29,12,37,7.818919,992.486486,4.994595,19,636.756757,6.684211,992.157895,3.405263
0,2019-12-30,12,37,7.432432,987.621622,6.756757,33,566.675676,7.439394,988.303030,6.748485


In [8]:
records_df[records_df.isna().any(axis=1)]

,date,month,len_day,temp_mean,press_mean,wind_mean,len_night,Y,night_temp_mean,night_press_mean,night_wind_mean
0,2017-01-01,1,37,4.910811,1001.648649,1.878378,0,1397.243243,NaN,NaN,NaN


In [19]:
def get_Data_n_records(df,num):
    X = []
    Y = []
    for i in range(len(df)):
        if i < num+1:
            pass
        else:
            X.append(df.iloc[i-num-1:i-1])
            Y.append (df.iloc[i].Y)
    return X,Y

get_Data_n_records(records_df,3)

([        date  month  len_day  temp_mean   press_mean  wind_mean  len_night  \
  0 2017-01-01      1       37   4.910811  1001.648649   1.878378          0   
  0 2017-01-02      1       37   6.035135  1000.918919   1.335135         11   
  0 2017-01-03      1       37   8.983784   999.756757   2.970270         14   
  
               Y  night_temp_mean  night_press_mean  night_wind_mean  
  0  1397.243243              NaN               NaN              NaN  
  0  1391.621622         3.863636       1000.727273         1.709091  
  0  1068.054054         5.828571       1000.285714         1.835714  ,
          date  month  len_day  temp_mean   press_mean  wind_mean  len_night  \
  0 2017-01-02      1       37   6.035135  1000.918919   1.335135         11   
  0 2017-01-03      1       37   8.983784   999.756757   2.970270         14   
  0 2017-01-04      1       37  10.167568   996.621622   3.432432         15   
  
               Y  night_temp_mean  night_press_mean  night_wind_mean 